In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from os import listdir

Parse text files into usable csv datasets

In [10]:
name = 'weck_jars'
file = open('data/text/{jars_doc}.txt'.format(jars_doc=name))
lines = file.readlines()

def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return zip(a, a)

value_list = []
for line in lines:
    entry = {}
    parsed_line = json.loads(line)
    for column, value in pairwise(parsed_line):
        entry[column] = value
    value_list.append(entry)

df = pd.DataFrame(value_list)
df.to_csv('./data/csv/{jars_doc}.csv'.format(jars_doc=name))

Merge all datasets into one Dataframe

In [3]:
df = pd.DataFrame()
for file in listdir('./data/csv/jars/'):
    current_df = pd.read_csv('./data/csv/jars/{csv_file}'.format(csv_file=file))
    df = pd.concat([df, current_df])

Clean Dataframe

In [4]:
# Drop columns, change SKU Ref. to Id, and remove duplicates by Id
df = df.drop(['Unnamed: 0', 'Units per pasteurizer', 'Technical drawing', 'Length', 'Width', 'Diameter'], axis=1)
df = df.rename(columns={'SKU reference': 'Id'})
df.drop_duplicates(subset=['Id'], inplace=True)
df.reset_index(drop=True, inplace=True)

Finish (part 1)

In [5]:
# Only use rows where Finish column is not null -> Finish determines the Affordance
df = df[df['Finish'].notna()]

Capacity

In [6]:
df['Capacity'] = df['Capacity'].str.replace(' ml', '')
df['Capacity'] = pd.to_numeric(df['Capacity'])

df.loc[(df['Capacity'] <= 250), 'Capacity 1'] = '<= 250 mL'
df.loc[(df['Capacity'] > 250) & (df['Capacity'] <= 500), 'Capacity 1'] = '250 - 500 mL'
df.loc[(df['Capacity'] > 500) & (df['Capacity'] <= 750), 'Capacity 1'] = '500 - 750 mL'
df.loc[(df['Capacity'] > 750) & (df['Capacity'] < 1000), 'Capacity 1'] = '750 - 1000 mL'
df.loc[df['Capacity'] >= 1000, 'Capacity 1'] = '>= 1000 mL'

df = df.drop(['Capacity'], axis=1)
df = df.rename(columns={'Capacity 1': 'Capacity'})

C:\Users\jagui\AppData\Local\Temp\ipykernel_1244\2151157840.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<= 250 mL' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['Capacity'] <= 250), 'Capacity 1'] = '<= 250 mL'


Weight

In [7]:
df['Weight'] = df['Weight'].str.replace(' grs', '')
df['Weight'] = pd.to_numeric(df['Weight'])

df.loc[(df['Weight'] <= 250), 'Weight 1'] = '<= 250 g'
df.loc[(df['Weight'] > 250) & (df['Weight'] <= 500), 'Weight 1'] = '250 - 500 g'
df.loc[(df['Weight'] > 500) & (df['Weight'] <= 750), 'Weight 1'] = '500 - 750 g'
df.loc[(df['Weight'] > 750) & (df['Weight'] < 1000), 'Weight 1'] = '750 - 1000 g'
df.loc[df['Weight'] >= 1000, 'Weight 1'] = '>= 1000 g'

df = df.drop(['Weight'], axis=1)
df = df.rename(columns={'Weight 1': 'Weight'})

C:\Users\jagui\AppData\Local\Temp\ipykernel_1244\121108913.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<= 250 g' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['Weight'] <= 250), 'Weight 1'] = '<= 250 g'


Material

In [8]:
material_nan = df[df['Material'].isna()]
for index in material_nan.index.values:
    color = df.loc[index,'Color']
    material = 'Plastic'
    if 'glass' in color:
        material = 'Glass'
    df.loc[index, 'Material'] = material

Height

In [9]:
df['Height'] = df['Height'].str.replace(' mm', '')
df['Height'] = df['Height'].str.replace(',', '.')
df['Height'] = pd.to_numeric(df['Height'])

df.loc[(df['Height'] <= 50), 'Height 1'] = '<= 50 mm'
df.loc[(df['Height'] > 50) & (df['Height'] <= 100), 'Height 1'] = '50 - 100 mm'
df.loc[(df['Height'] > 100) & (df['Height'] <= 150), 'Height 1'] = '100 - 150 mm'
df.loc[(df['Height'] > 150) & (df['Height'] < 200), 'Height 1'] = '150 - 200 mm'
df.loc[df['Height'] >= 200, 'Height 1'] = '>= 200 mm'

df = df.drop(['Height'], axis=1)
df = df.rename(columns={'Height 1': 'Height'})

C:\Users\jagui\AppData\Local\Temp\ipykernel_1244\3582645053.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<= 50 mm' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['Height'] <= 50), 'Height 1'] = '<= 50 mm'


Color

In [10]:
df['Color'].unique()
for index in df.index.values:
    color = df.loc[index, 'Color']
    final_color = 'White'
    if 'transparent' in color:
        final_color = 'Transparent'
    elif 'black' in color:
        final_color = 'Black'
    df.loc[index, 'Color'] = final_color

Shape

In [11]:
# Only use rows where Shape column is not null -> Only 2 null rows
df = df[df['Shape'].notna()]
df['Shape'] = df['Shape'].str.capitalize()

Finish (part 2)

In [12]:
for index in df.index.values:
    finish = df.loc[index,'Finish']
    type = 'Special'
    if ('TO' in finish) | ('Twist Off' in finish) | ('PP' in finish) | ('Screw' in finish) | ('Rosco' in finish):
        type = 'Twist Off'
    elif ('Latex' in finish) | ('Swing' in finish):
        type = 'Swing Top'
    elif 'Weck' in finish:
        type = 'Rubber Ring'
    elif 'Thread' in finish:
        type = 'Sprayer'
    elif 'Wiss' in finish:
        type = 'Wiss'
    df.loc[index, 'Finish'] = type
df.drop(df[df['Finish'] == 'Special'].index, inplace=True)

Basic Affordances

In [13]:
df['Grasp'] = True
df['Swing'] = False
for index in df.index.values:
    if df.loc[index, 'Finish'] == 'Swing Top':
        df.loc[index, 'Swing'] = True

df = df.drop(['Id'], axis=1)
df.reset_index(drop=True, inplace=True)

In [14]:
df.to_csv('./data/csv/jars.csv')